# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
from distutils.version import StrictVersion
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

C:\mgr\tensor\models\research\object_detection\utils\visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "c:\users\daniel żukowski\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\daniel żukowski\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\daniel żukowski\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\daniel żukowski\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_i

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
# MODEL_FILE = MODEL_NAME + '.tar.gz'
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
# PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_FROZEN_GRAPH = 'C://mgr//mgr_kask//grafy_wytrenowane//output_inference_graph_rcnn.pb//frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'C://mgr//mgr_kask//grafy_wytrenowane//object-detection.pbtxt'

NUM_CLASSES = 1

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [9]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'C:\\mgr\\drive-download-20180905T195237Z-001\\DSCN9955-testFrames'
IMAGE_PATHS = []
for root, dirs, files in os.walk(PATH_TO_TEST_IMAGES_DIR):  
    for filename in files:
        if str(filename).__contains__('.jpg'):
            IMAGE_PATHS.append(filename)
            print(filename)
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, i) for i in IMAGE_PATHS ]
print(TEST_IMAGE_PATHS)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

frame1166.jpg
frame1167.jpg
frame1168.jpg
frame1169.jpg
frame117.jpg
frame1170.jpg
frame1172.jpg
frame1173.jpg
frame1174.jpg
frame1175.jpg
frame1176.jpg
frame1177.jpg
frame1178.jpg
frame1179.jpg
frame118.jpg
frame1180.jpg
frame1181.jpg
frame1182.jpg
frame1183.jpg
frame1184.jpg
frame1185.jpg
frame1186.jpg
frame1187.jpg
frame1188.jpg
frame1189.jpg
frame119.jpg
frame1190.jpg
frame1191.jpg
frame1192.jpg
frame1193.jpg
frame1194.jpg
frame1195.jpg
frame1196.jpg
frame1197.jpg
frame1198.jpg
frame1199.jpg
frame12.jpg
frame120.jpg
frame1200.jpg
frame1201.jpg
frame1202.jpg
frame1203.jpg
frame1204.jpg
frame1205.jpg
frame1206.jpg
frame1207.jpg
frame1209.jpg
frame121.jpg
frame1301.jpg
frame1302.jpg
frame1303.jpg
frame1304.jpg
frame1305.jpg
frame1306.jpg
frame1307.jpg
frame1308.jpg
frame1309.jpg
frame131.jpg
frame1310.jpg
frame1311.jpg
frame1312.jpg
frame1313.jpg
frame1314.jpg
frame1315.jpg
frame1316.jpg
frame1317.jpg
frame1318.jpg
frame1319.jpg
frame132.jpg
frame1320.jpg
['C:\\mgr\\drive-download-201

In [10]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [11]:
def transformOutputToEvaluateFormat(output_dict, image):
    im_width, im_height = image.size
    num_detections = int(output_dict['num_detections'])
    detection_boxes = output_dict['detection_boxes']
    detection_scores = output_dict['detection_scores']
    detection_classes =   output_dict['detection_classes']
    
    classString = 'bee'
    objectLines = []
    
    for i in range(0, num_detections):
        ymin=detection_boxes[i][0]
        xmin=detection_boxes[i][1]
        ymax=detection_boxes[i][2]
        xmax=detection_boxes[i][3]
         #ymin, xmin, ymax, xmax = box
#         (left, right, top, bottom) = (int(xmin * im_width), int(xmax * im_width), int(ymin * im_height), int(ymax * im_height))
#         print('absolute coordinates (left, top, right, bottom): ', left, top, right, bottom)
#         single_row = (classString, detection_scores[i],  left, top, right, bottom)
        single_row = (classString, detection_scores[i],  xmin, ymin, xmax, ymax)
        objectLines.append(single_row)
#         print(single_row)
    print(objectLines)
    return objectLines
    

In [12]:
def saveDetectionToFolder(image_path ,linesArray, savepath):
    f = open( os.path.join(savepath, os.path.splitext(os.path.basename(image_path))[0] + ".txt"),"w+")
    for i in linesArray:
        f.write(' '.join(str(x) for x in i) + '\n')
    

In [13]:
DETECTIONS_PATH='.//detections//'

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  print(os.path.splitext(os.path.basename(image_path))[0] + ".txt")
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
#   print(tf.metrics.recall( output_dict['detection_classes'],output_dict['detection_boxes']))
#   print(output_dict)
  detectionsLines = transformOutputToEvaluateFormat(output_dict, image)
  saveDetectionToFolder(image_path, detectionsLines, DETECTIONS_PATH)
  print("========================")
    
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

frame1166.txt
[('bee', 0.9992894, 0.55922014, 0.7963183, 0.62612265, 0.88366306), ('bee', 0.99922955, 0.348271, 0.7706812, 0.43565428, 0.87656695)]
frame1167.txt
[('bee', 0.99938536, 0.34793586, 0.7713623, 0.43603438, 0.87582135), ('bee', 0.9983164, 0.56268895, 0.79493976, 0.61747754, 0.88755447), ('bee', 0.99628747, 0.3769603, 0.11271238, 0.4982473, 0.2950259), ('bee', 0.6495886, 0.9327944, 0.44922093, 1.0, 0.6569918)]
frame1168.txt
[('bee', 0.99937195, 0.34703234, 0.7703487, 0.4343907, 0.8789169), ('bee', 0.99837166, 0.3747895, 0.11205508, 0.4791763, 0.25792667), ('bee', 0.9982254, 0.55795294, 0.796621, 0.6149855, 0.88604134), ('bee', 0.9309409, 0.3419373, 0.024077237, 0.4442129, 0.12746817), ('bee', 0.7388126, 0.6640208, 0.76760507, 0.7597867, 0.8874054), ('bee', 0.5316118, 0.9298314, 0.5416824, 0.99983096, 0.7062297)]
frame1169.txt
[('bee', 0.99802196, 0.34988427, 0.76697576, 0.41708803, 0.8845258), ('bee', 0.99714524, 0.5552981, 0.79706573, 0.61267185, 0.884158), ('bee', 0.9968275

c:\users\daniel żukowski\appdata\local\programs\python\python36\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


frame1186.txt
[('bee', 0.9976259, 0.39508426, 0.083009385, 0.4963714, 0.26614743), ('bee', 0.9895711, 0.60453534, 0.7323319, 0.6781039, 0.8782398), ('bee', 0.95545834, 0.3195587, 0.7678901, 0.42214867, 0.8923432), ('bee', 0.86290187, 0.5270212, 0.5844653, 0.5824874, 0.7230944), ('bee', 0.8432588, 0.65910023, 0.7529002, 0.7237838, 0.8952412)]
frame1187.txt
[('bee', 0.9987404, 0.40040904, 0.09665916, 0.49523312, 0.27857316), ('bee', 0.9889682, 0.6050127, 0.73142976, 0.6745587, 0.8786471), ('bee', 0.9502644, 0.31733438, 0.7667819, 0.427308, 0.89017093), ('bee', 0.84966624, 0.65841526, 0.74891484, 0.73001105, 0.88730556)]
frame1188.txt
[('bee', 0.9987093, 0.40073082, 0.13267033, 0.49478552, 0.30191332), ('bee', 0.9900019, 0.3187861, 0.77047175, 0.42660192, 0.88800424), ('bee', 0.977653, 0.60629624, 0.7326242, 0.67496973, 0.8723136), ('bee', 0.9733236, 0.6584785, 0.75590146, 0.7305651, 0.885872)]
frame1189.txt
[('bee', 0.99878424, 0.40077603, 0.1350619, 0.49931926, 0.3108797), ('bee', 0.995